# Editing the CTSM paramfiles
- some typical examples

In [3]:
import numpy as np
import xarray as xr
import os
import netCDF4

In [1]:
#this is the base paramfile that you will copy and edit
basefile = '/glade/p/cesm/cseg/inputdata/lnd/clm2/paramdata/ctsm51_params.c210507.nc'

#important that your paramfile match the version of CLM that you are using
# consider examining bld/namelist_files/namelist_defaults_ctsm.xml

### examine a pft file

In [5]:
ds = xr.open_dataset(basefile)

In [6]:
ds.kmax

<xarray.DataArray 'kmax' (segment: 4, pft: 79)>
array([[0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.000000e-08,
        2.000000e-08, 2.000000e-08],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.000000e-08,
        2.000000e-08, 2.000000e-08],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.000000e-08,
        2.000000e-08, 2.000000e-08],
       [0.000000e+00, 1.602140e-08, 2.234574e-08, ..., 2.000000e-08,
        2.000000e-08, 2.000000e-08]])
Coordinates:
    pftname  (pft) |S40 ...
  * segment  (segment) |S40 b'sunlit                                  ' ... b...
Dimensions without coordinates: pft
Attributes:
    units:      mm h2o (transpired)/mm h2o (water potential gradient)/sec
    long_name:  plant segment max conductance

In [12]:
ds.medlynslope

<xarray.DataArray 'medlynslope' (pft: 79)>
array([0.  , 2.35, 2.35, 2.35, 4.12, 4.12, 4.45, 4.45, 4.45, 4.7 , 4.7 , 4.7 ,
       2.22, 5.25, 1.62, 5.79, 5.79, 1.79, 1.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 1.79, 1.79, 5.79, 5.79, 1.79,
       1.79, 1.79, 1.79, 1.79, 1.79, 5.79, 5.79])
Coordinates:
    pftname  (pft) |S40 b'not_vegetated                           ' ... b'irr...
Dimensions without coordinates: pft
Attributes:
    long_name:  Medlyn slope of conductance-photosynthesis relationship
    units:      umol H2O/umol CO2
    comment:    Values come from the values used in the CABLE model

In [13]:
ds.jmaxb0

<xarray.DataArray 'jmaxb0' ()>
array(0.0311)
Attributes:
    long_name:  Baseline proportion of nitrogen allocated for electron transport
    units:      J

## edit a paramfile
 - avoiding xarray for writing the new files, for some reason paramfiles written with xarray tend to crash the model

### jmaxb0, increase to 0.05

In [22]:
## CREATE the new paramfile
newfile = 'jmaxb0_5e-2.nc'
cmd = 'cp '+basefile+' '+newfile
os.system(cmd)

0

In [25]:
param = 'jmaxb0'
dset = netCDF4.Dataset(newfile,'r+')
dset[param][:] = 0.05
dset.close()

In [26]:
xr.open_dataset(newfile)['jmaxb0']

<xarray.DataArray 'jmaxb0' ()>
array(0.05)
Attributes:
    long_name:  Baseline proportion of nitrogen allocated for electron transport
    units:      J

### kmax, double all values
 - note that the shapes of the various parameters vary
 - some are scalar, some have a pft-dimension, and some have pft and plant-segment dimensions
 - easist to read in the example from basefile, and edit accordingly 

In [33]:
## CREATE the new paramfile
newfile = 'kmax_2x.nc'
cmd = 'cp '+basefile+' '+newfile
os.system(cmd)

0

In [35]:
param = 'kmax'
kmax = xr.open_dataset(basefile)[param].values
newkmax = kmax*2

In [36]:
dset = netCDF4.Dataset(newfile,'r+')
dset[param][:] = newkmax
dset.close()

In [37]:
xr.open_dataset(newfile)['kmax']

<xarray.DataArray 'kmax' (segment: 4, pft: 79)>
array([[0.000000e+00, 3.204280e-08, 4.469149e-08, ..., 4.000000e-08,
        4.000000e-08, 4.000000e-08],
       [0.000000e+00, 3.204280e-08, 4.469149e-08, ..., 4.000000e-08,
        4.000000e-08, 4.000000e-08],
       [0.000000e+00, 3.204280e-08, 4.469149e-08, ..., 4.000000e-08,
        4.000000e-08, 4.000000e-08],
       [0.000000e+00, 3.204280e-08, 4.469149e-08, ..., 4.000000e-08,
        4.000000e-08, 4.000000e-08]])
Coordinates:
    pftname  (pft) |S40 ...
  * segment  (segment) |S40 b'sunlit                                  ' ... b...
Dimensions without coordinates: pft
Attributes:
    units:      mm h2o (transpired)/mm h2o (water potential gradient)/sec
    long_name:  plant segment max conductance

### medlynslope
- substituting a value for just one pft

In [48]:
p = xr.open_dataset(basefile)
mypft = 'broadleaf_deciduous_temperate_tree'
ix    = np.array([mypft in str(n) for n in p.pftname.values])  #index vector for mypft

In [53]:
param = 'medlynslope'
m = p[param].values
m[ix] = 6
m

array([0.        , 2.3499999 , 2.3499999 , 2.3499999 , 4.11999989,
       4.11999989, 4.44999981, 6.        , 4.44999981, 4.69999981,
       4.69999981, 4.69999981, 2.22000003, 5.25      , 1.62      ,
       5.78999996, 5.78999996, 1.78999996, 1.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 5.78999996, 5.78999996, 5.78999996,
       5.78999996, 5.78999996, 1.78999996, 1.78999996, 5.78999996,
       5.78999996, 1.78999996, 1.78999996, 1.78999996, 1.78999

In [54]:
newfile = 'medlynslope_BDT_6.nc'
cmd = 'cp '+basefile+' '+newfile
os.system(cmd)
dset = netCDF4.Dataset(newfile,'r+')
dset[param][:] = m
dset.close()

In [55]:
xr.open_dataset(newfile)[param]

<xarray.DataArray 'medlynslope' (pft: 79)>
array([0.  , 2.35, 2.35, 2.35, 4.12, 4.12, 4.45, 6.  , 4.45, 4.7 , 4.7 , 4.7 ,
       2.22, 5.25, 1.62, 5.79, 5.79, 1.79, 1.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79,
       5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 5.79, 1.79, 1.79, 5.79, 5.79, 1.79,
       1.79, 1.79, 1.79, 1.79, 1.79, 5.79, 5.79])
Coordinates:
    pftname  (pft) |S40 ...
Dimensions without coordinates: pft
Attributes:
    long_name:  Medlyn slope of conductance-photosynthesis relationship
    units:      umol H2O/umol CO2
    comment:    Values come from the values used in the CABLE model